In [1]:
knitr::opts_chunk$set(echo = TRUE)

# Install the required packages

In [2]:
options(repos = c(CRAN = "https://cran.rstudio.com/"))

if (!require("ggplot2")) install.packages("ggplot2")
if (!require("e1071")) install.packages("e1071")
if (!require("tidyverse")) install.packages("tidyverse")
if (!require("dplyr")) install.packages("dplyr")
if (!require("effsize")) install.packages("effsize")

Loading required package: ggplot2

Loading required package: e1071

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘e1071’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’


Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: effsize

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = 

In [3]:
install.packages("car")
install.packages('cowplot')
install.packages('grid')
install.packages("png")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘grid’ is a base package, and should not be updated”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
library("car")
library('cowplot')
library('grid')
library("png")

library(e1071)
library(effsize)

library(ggplot2)
library(tidyverse)
library(dplyr)


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




# Define some global variables

In [96]:
ON_DEVICE <- "on_device"
REMOTE <- "remote"
LENGTHS <- c(100, 500, 1000)
LENGTH_LABELS <- c("short", "medium", "long")
LENGTH_MAP <- setNames(LENGTHS, LENGTH_LABELS)  # Map lengths to labels
# Color mapping for On-Device and Remote using R's native colors
COLOR_MAP <- c(on_device = "coral",
               remote = "lightblue")

ENERGY <- "energy_usage_J"
TIME <- "execution_time"
CPU <- "cpu_usage"
GPU <- "gpu_usage"
MEMORY <- "memory_usage"

METRICS <- c(ENERGY, TIME, CPU, GPU, MEMORY)

AXIS_LABELS <- list(
  energy_usage_J = "Energy Usage (J)",
  execution_time = "Execution Time (%)",
  cpu_usage = "CPU Usage (%)",
  gpu_usage = "GPU Usage (%)",
  memory_usage = "Memory Usage (%)"
)


DENSITY_FOLDER <- "density_plots"
VIOLIN_FOLDER <- "violin_plots"
QQ_FOLDER <- "qq_plots"
SCATTER_FOLDER <- "scatter_plots"

PLOT_FOLDERS <- c(DENSITY_FOLDER, VIOLIN_FOLDER, QQ_FOLDER, SCATTER_FOLDER)

WIDTH = 8
HEIGHT = 12
FONT_MULTIPLIER = 3

LLM_NAMES <- c(
  "Qwen 2 1.5B" = "qwen2:1.5b",
  "Gemma 1.1 2B" = "gemma:2b",
  "Phi 3 3B" = "phi3:3.8b",
  "Qwen 2 7B" = "qwen2:7b",
  "Gemma 1.1 7B" = "gemma:7b",
  "Mistral 0.3 7B" = "mistral:7b",
  "Llama 3.1 8B" = "llama3.1:8b"
)

LLM_SHORT_LABELS <- c(
  "Qwen 2 1.5B" = "Q1.5",
  "Gemma 1.1 2B" = "G2",
  "Phi 3 3B" = "P3",
  "Qwen 2 7B" = "Q7",
  "Gemma 1.1 7B" = "G7",
  "Mistral 0.3 7B" = "M7",
  "Llama 3.1 8B" = "L8"
)

LLM_COLOR_MAP <- c(
  "#ff0000",  # Red
  "#ff8000",  # Orange
  "#ffff00",  # Yellow
  "#00ff00",  # Green
  "#00ffff",  # Cyan (Light Blue)
  "#0000ff",  # Blue
  "#8000ff"   # Violet
)


# Read the run table file

In [6]:
dataset <- read.csv("./run_table.csv")
print(head(dataset))


             X__run_id X__done       model    method length               topic
1  run_32_repetition_9    DONE    qwen2:7b on_device   1000         World War I
2 run_34_repetition_27    DONE    qwen2:7b    remote    500       Elvis Presley
3 run_10_repetition_17    DONE    gemma:2b    remote    500     John F. Kennedy
4 run_21_repetition_14    DONE   phi3:3.8b    remote    100           Tom Brady
5  run_4_repetition_11    DONE llama3.1:8b    remote    500        Adolf Hitler
6 run_41_repetition_10    DONE  mistral:7b    remote   1000 William Shakespeare
  execution_time cpu_usage gpu_usage memory_usage codecarbon__energy_consumed
1      49.812157     3.151    95.507       73.341                0.0001414828
2      15.413524     2.888     0.196       51.100                0.0000152000
3       9.873243     5.033     0.448       55.267                0.0000058000
4       9.877062    10.767     0.459       52.367                0.0000058000
5      15.385836     4.588     0.225       53.200 

# H1 Analysis.

## Define a Function to Remove Outliers using IQR method (works on multiple columns of a dataset)

In [7]:
# Function to remove outliers using the IQR method
remove_outliers <- function(data, columns) {
  # Initialize filtered data as the original data
  filtered_data <- data

  # Loop over each column and apply IQR outlier filtering
  for (column in columns) {
    Q1 <- quantile(filtered_data[[column]], 0.25, na.rm = TRUE)
    Q3 <- quantile(filtered_data[[column]], 0.75, na.rm = TRUE)
    IQR_value <- Q3 - Q1
    lower_bound <- Q1 - 1.5 * IQR_value
    upper_bound <- Q3 + 1.5 * IQR_value

    # Filter rows where the column values are within bounds
    filtered_data <- filtered_data %>%
      filter(filtered_data[[column]] >= lower_bound & filtered_data[[column]] <= upper_bound)
  }

  return(filtered_data)
}

## Filter dataset into 3 blocks & 2 treatments while removing outliers based on `ENERGY` column
- 2 Treatments:
  - `on_device`
  - `remote`

- 3 Blocks (blocking factor: requested LLM content length):
  - Short (100 words)
  - Medium (500 words)
  - Long (1000 words)

--> 6 subsets

In [8]:
# Store subsets in a named list for easy access
subset_data <- list()

# Loop over method and length combinations, using labels
for (method in c(ON_DEVICE, REMOTE)) {
  for (label in names(LENGTH_MAP)) {
    length_value <- LENGTH_MAP[[label]]
    subset_name <- paste(method, label, sep = "_")  # e.g., "on_device_short"

    # Filter the dataset
    filtered_data <- dataset %>%
      filter(method == !!method, length == length_value)

    # Call the remove_outliers function with the filtered data and the metrics columns
    subset_data[[subset_name]] <- remove_outliers(filtered_data, METRICS)
  }
}

# Access example: on-device data for "short" length
# on_device_data_short <- subset_data[["on_device_short"]]

## Table: Mean, Median, and Standard Deviation (SD) of Energy Usage and Performance Metrics for Fetching LLM Content On-Device vs. Remote Across Varying Content Lengths

In [9]:
# Prepare LaTeX table header
cat("\\begin{table*}[htbp]\n")
cat("    \\centering\n")
cat("    \\caption{Mean, Median, and Standard Deviation (SD) of Energy Usage and Performance Metrics for Fetching LLM Content On-Device vs. Remote Across Varying Content Lengths}\n")
cat("    \\scalebox{0.8}{\n")
cat("    \\begin{tabular}{|l|l|ccc|ccc|ccc|ccc|ccc|}\n")
cat("        \\hline\n")
cat("        \\multirow{2}{*}{\\textbf{Content Length}} & \\multirow{2}{*}{\\textbf{Treatment}} & \\multicolumn{3}{c|}{\\textbf{Energy Usage (Joule)}} & \\multicolumn{3}{c|}{\\textbf{Execution Time (second)}} & \\multicolumn{3}{c|}{\\textbf{Average CPU Usage (\\%)}} & \\multicolumn{3}{c|}{\\textbf{Average GPU Usage (\\%)}} & \\multicolumn{3}{c|}{\\textbf{Average Memory Usage (\\%)}} \\\\ \n")
cat("        \\cline{3-17}\n")
cat("        & & \\textbf{Mean} & \\textbf{Median} & \\textbf{SD} & \\textbf{Mean} & \\textbf{Median} & \\textbf{SD} & \\textbf{Mean} & \\textbf{Median} & \\textbf{SD} & \\textbf{Mean} & \\textbf{Median} & \\textbf{SD} & \\textbf{Mean} & \\textbf{Median} & \\textbf{SD} \\\\ \n")
cat("        \\hline\n")

# Loop over lengths first and then methods to populate rows
for (label in names(LENGTH_MAP)) {
  length_value <- LENGTH_MAP[[label]]
  for (method in c(ON_DEVICE, REMOTE)) {
    subset_name <- paste(method, label, sep = "_")

    # Extract the subset data
    data <- subset_data[[subset_name]]

    # Calculate mean, median, and SD for each metric
    energy_mean <- mean(data[[ENERGY]], na.rm = TRUE)
    energy_median <- median(data[[ENERGY]], na.rm = TRUE)
    energy_sd <- sd(data[[ENERGY]], na.rm = TRUE)

    time_mean <- mean(data[[TIME]], na.rm = TRUE)
    time_median <- median(data[[TIME]], na.rm = TRUE)
    time_sd <- sd(data[[TIME]], na.rm = TRUE)

    cpu_mean <- mean(data[[CPU]], na.rm = TRUE)
    cpu_median <- median(data[[CPU]], na.rm = TRUE)
    cpu_sd <- sd(data[[CPU]], na.rm = TRUE)

    gpu_mean <- mean(data[[GPU]], na.rm = TRUE)
    gpu_median <- median(data[[GPU]], na.rm = TRUE)
    gpu_sd <- sd(data[[GPU]], na.rm = TRUE)

    memory_mean <- mean(data[[MEMORY]], na.rm = TRUE)
    memory_median <- median(data[[MEMORY]], na.rm = TRUE)
    memory_sd <- sd(data[[MEMORY]], na.rm = TRUE)

    # Print row in LaTeX format
    if (method == ON_DEVICE) {
      cat(sprintf("        \\textbf{%s (%d words)} & \\textbf{On-Device} & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f \\\\ \n",
                  tools::toTitleCase(label), length_value,
                  energy_mean, energy_median, energy_sd,
                  time_mean, time_median, time_sd,
                  cpu_mean, cpu_median, cpu_sd,
                  gpu_mean, gpu_median, gpu_sd,
                  memory_mean, memory_median, memory_sd))
    } else {
      cat(sprintf("        & \\textbf{Remote} & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f \\\\ \n",
                  energy_mean, energy_median, energy_sd,
                  time_mean, time_median, time_sd,
                  cpu_mean, cpu_median, cpu_sd,
                  gpu_mean, gpu_median, gpu_sd,
                  memory_mean, memory_median, memory_sd))
    }

    # Add \hline after Remote entry except for the last label
    if (method == REMOTE && label != tail(names(LENGTH_MAP), n = 1)) {
      cat("        \\hline\n")
    }
  }
}

# Close LaTeX table
cat("    \\hline\n")
cat("    \\end{tabular}\n")
cat("    }\n")
cat("    \\label{table:performance_metrics}\n")
cat("\\end{table*}\n")


\begin{table*}[htbp]
    \centering
    \caption{Mean, Median, and Standard Deviation (SD) of Energy Usage and Performance Metrics for Fetching LLM Content On-Device vs. Remote Across Varying Content Lengths}
    \scalebox{0.8}{
    \begin{tabular}{|l|l|ccc|ccc|ccc|ccc|ccc|}
        \hline
        \multirow{2}{*}{\textbf{Content Length}} & \multirow{2}{*}{\textbf{Treatment}} & \multicolumn{3}{c|}{\textbf{Energy Usage (Joule)}} & \multicolumn{3}{c|}{\textbf{Execution Time (second)}} & \multicolumn{3}{c|}{\textbf{Average CPU Usage (\%)}} & \multicolumn{3}{c|}{\textbf{Average GPU Usage (\%)}} & \multicolumn{3}{c|}{\textbf{Average Memory Usage (\%)}} \\ 
        \cline{3-17}
        & & \textbf{Mean} & \textbf{Median} & \textbf{SD} & \textbf{Mean} & \textbf{Median} & \textbf{SD} & \textbf{Mean} & \textbf{Median} & \textbf{SD} & \textbf{Mean} & \textbf{Median} & \textbf{SD} & \textbf{Mean} & \textbf{Median} & \textbf{SD} \\ 
        \hline
        \textbf{Short (100 words)} & \textbf{On-Dev

# Normality Check

## Visualize data for manual normality check

In [10]:
# Remove the density folder if it exists, including its contents
system("rm -rf density_plots")

In [11]:
# Loop through the plot folders and create them if they don't exist
for (folder in PLOT_FOLDERS) {
  if (!dir.exists(folder)) {
    dir.create(folder)
  }
}


### Violin and Density Plots

In [63]:
# Re-usable functions to make density plots and violin plots
density_plot <- function(dataset, title, x, y, show_legend = TRUE) {
  ggplot(dataset, aes(x = Values, fill = Distribution)) +
    geom_density(alpha = 0.5) +
    scale_fill_manual(values = COLOR_MAP) +  # Use COLOR_MAP for colors
    labs(
      title = title,
      x = x,
      y = y
    ) +
    theme_minimal() +
    theme(
      legend.position = ifelse(show_legend, "right", "none"),  # Control legend display
      legend.title = element_blank(),
      legend.text = element_text(size = rel(FONT_MULTIPLIER)),  # Increase legend text size
      axis.title = element_text(size = rel(FONT_MULTIPLIER)),
      axis.text = element_text(size = rel(FONT_MULTIPLIER)),
      plot.title = element_text(size = rel(1.1 * FONT_MULTIPLIER))
    )
}

violin_plot <- function(dataset, title, x, y, show_legend = TRUE) {
  ggplot(dataset, aes(x = Distribution, y = Values, fill = Distribution)) +
    geom_violin(alpha = 0.5) +
    geom_boxplot(width = 0.045, outlier.shape = NA) +
    scale_fill_manual(values = COLOR_MAP) +  # Use COLOR_MAP for colors
    labs(
      title = title,
      x = x,
      y = y
    ) +
    theme_minimal() +
    theme(
      legend.position = ifelse(show_legend, "right", "none"),  # Control legend display
      legend.title = element_blank(),
      legend.text = element_text(size = rel(FONT_MULTIPLIER)),  # Increase legend text size
      axis.title = element_text(size = rel(FONT_MULTIPLIER)),
      axis.text = element_text(size = rel(FONT_MULTIPLIER)),
      plot.title = element_text(size = rel(1.1 * FONT_MULTIPLIER))
    )
}

In [70]:
generate_plots_for_metric <- function(metric_name) {
  # Set up directories for the specific metric if they don't exist
  density_dir <- paste0(DENSITY_FOLDER, "/", metric_name)
  violin_dir <- paste0(VIOLIN_FOLDER, "/", metric_name)

  if (!dir.exists(density_dir)) {
    dir.create(density_dir, recursive = TRUE)
  }
  if (!dir.exists(violin_dir)) {
    dir.create(violin_dir, recursive = TRUE)
  }

  # List to store each plot for grid arrangement
  density_plots <- list()
  violin_plots <- list()

  # Collect all values across content lengths for consistent axis limits
  all_values <- unlist(lapply(names(LENGTH_MAP), function(label) {
    c(
      subset_data[[paste(ON_DEVICE, label, sep = "_")]][[metric_name]],
      subset_data[[paste(REMOTE, label, sep = "_")]][[metric_name]]
    )
  }))

  # Loop over content lengths to create individual density and violin plots
  for (label in names(LENGTH_MAP)) {
    length_value <- LENGTH_MAP[[label]]

    # Create combined data for On-Device and Remote for this length
    combined_data <- bind_rows(
      subset_data[[paste(ON_DEVICE, label, sep = "_")]] %>%
        mutate(Distribution = ON_DEVICE, Values = .[[metric_name]]),
      subset_data[[paste(REMOTE, label, sep = "_")]] %>%
        mutate(Distribution = REMOTE, Values = .[[metric_name]])
    )

    # Determine whether to show legend based on label (only for the last plot)
    show_legend <- FALSE

    # Create density plot
    plot_title_density <- paste(tools::toTitleCase(label), " (", length_value, ")", sep = "")
    density_plot_obj <- density_plot(combined_data, plot_title_density, AXIS_LABELS[[metric_name]], "Density", show_legend)
    density_plot_obj_for_combined <- density_plot(combined_data, plot_title_density, AXIS_LABELS[[metric_name]],
                                                  ifelse(label == "short", "Density", ""), show_legend)

    # Save individual density plot without x-axis limit for auto-adjusted scaling
    ggsave(filename = paste0(density_dir, "/density_plot_", label, ".pdf"),
           plot = density_plot_obj,
           width = WIDTH, height = HEIGHT)

    # Add x-axis limits for consistent axis in the combined plot
    density_plots[[label]] <- density_plot_obj_for_combined + xlim(range(all_values, na.rm = TRUE))

    # Create violin plot
    plot_title_violin <- paste(tools::toTitleCase(label), " (", length_value, ")", sep = "")
    violin_plot_obj <- violin_plot(combined_data, plot_title_violin, "", AXIS_LABELS[[metric_name]], show_legend)
    violin_plot_obj_for_combined <- violin_plot(combined_data, plot_title_violin, "",
                                                ifelse(label == "short", AXIS_LABELS[[metric_name]], ""), show_legend)

    # Save individual violin plot
    ggsave(filename = paste0(violin_dir, "/violin_plot_", label, ".pdf"),
           plot = violin_plot_obj,
           width = WIDTH, height = HEIGHT)

    # Add y-axis limits for consistent axis in the combined plot
    violin_plots[[label]] <- violin_plot_obj_for_combined + ylim(range(all_values, na.rm = TRUE))
  }

  # Arrange all density plots in a 1-row grid with consistent x-axis limits
  combined_density_plot <- plot_grid(density_plots$short, density_plots$medium, density_plots$long, nrow = 1)
  ggsave(paste0(density_dir, "/combined_density_plots_", metric_name, ".pdf"), combined_density_plot, width = WIDTH * length(LENGTH_MAP), height = HEIGHT)

  # Arrange all violin plots in a 1-row grid with consistent y-axis limits
  combined_violin_plot <- plot_grid(violin_plots$short, violin_plots$medium, violin_plots$long, nrow = 1)
  ggsave(paste0(violin_dir, "/combined_violin_plots_", metric_name, ".pdf"), combined_violin_plot, width = WIDTH * length(LENGTH_MAP), height = HEIGHT)
}


In [91]:
# Loop through each metric and generate plots
for (metric_name in METRICS) {
  generate_plots_for_metric(metric_name)
}

### Special Violin Plot for Each LLMs
- On-device
- 7 LLMs
- 3 content lengths
--> to qualitatively judge their spreads

In [103]:
filtered_llm_data <- list()

# Loop over each LLM
for (llm_key in names(LLM_NAMES)) {
  llm_name <- LLM_NAMES[[llm_key]]  # Get the full LLM name
  for (method in c(ON_DEVICE, REMOTE)) {
    for (label in names(LENGTH_MAP)) {
      length_value <- LENGTH_MAP[[label]]
      subset_name <- paste(method, label, sep = "_")  # e.g., "on_device_short"

      # Access the relevant data subset for the method and length
      if (subset_name %in% names(subset_data)) {
        filtered_data <- subset_data[[subset_name]]

        # Filter for the specific LLM based on the "model" column
        llm_filtered_data <- filtered_data %>%
          filter(model == tolower(gsub(" ", ":", llm_name)))  # Convert to match the model format in data

        # Store the result in the filtered_llm_data list
        filtered_llm_data[[paste(llm_name, label, method, sep = "_")]] <- llm_filtered_data
      }
    }
  }
}

[1] 42


In [143]:
generate_violin_plots_each_llm <- function(filtered_llm_data, metric_name) {
  # Set up directories for the specific metric if they don't exist
  violin_dir <- paste0(VIOLIN_FOLDER, "/", metric_name)

  if (!dir.exists(violin_dir)) {
    dir.create(violin_dir, recursive = TRUE)
  }

  # List to store each plot for grid arrangement
  violin_plots <- list()

  # Collect all values across content lengths for consistent axis limits
  all_values <- unlist(lapply(names(LENGTH_MAP), function(label) {
    sapply(names(LLM_NAMES), function(llm_key) {
      on_device_data <- subset_data[[paste(ON_DEVICE, label, sep = "_")]] %>%
        filter(model == tolower(gsub(" ", ":", LLM_NAMES[[llm_key]]))) %>%
        pull(!!sym(metric_name))
      return(on_device_data)
    })
  }))

  # Loop over content lengths to create individual density and violin plots
  for (label in names(LENGTH_MAP)) {
    length_value <- LENGTH_MAP[[label]]

    # Initialize a list to collect all On-Device data for this length
    combined_data_list <- list()

    # Loop through each LLM and gather On-Device data
    for (llm_key in names(LLM_NAMES)) {
      llm_name <- LLM_NAMES[[llm_key]]

      # Access On-Device data for the specific LLM
      on_device_data <- subset_data[[paste(ON_DEVICE, label, sep = "_")]] %>%
        filter(model == tolower(gsub(" ", ":", llm_name))) %>%
        mutate(Distribution = ON_DEVICE, Values = .[[metric_name]], LLM = llm_key)

      # Add the On-Device data to the list
      if (nrow(on_device_data) > 0) {
        combined_data_list[[llm_key]] <- on_device_data
      }
    }

    # Combine all On-Device data into a single dataframe
    combined_data <- bind_rows(combined_data_list)

    # Set LLM order as specified in LLM_NAMES, ensuring factor levels match keys in LLM_COLOR_MAP
    combined_data$LLM <- factor(combined_data$LLM, levels = names(LLM_NAMES))

    # Create plot title for the current content length
    plot_title_violin <- paste(tools::toTitleCase(label), " (", length_value, ")", sep = "")

    # Generate the violin plot
    violin_plot_obj_for_combined <- ggplot(combined_data, aes(x = LLM, y = Values, fill = LLM)) +
      geom_violin(position = "dodge", alpha = 0.5, trim = FALSE) +
      geom_boxplot(width = 0.07, color = "black", alpha = 0.6, outlier.shape = NA) +
      scale_fill_manual(values = LLM_COLOR_MAP) +  # Directly use LLM_COLOR_MAP without subsetting
      labs(
        title = plot_title_violin,
        x = "",
        y = ifelse(label == "short", AXIS_LABELS[[metric_name]], "")  # Display y-axis label only for the first plot
      ) +
      theme_minimal() +
      theme(
        legend.position = "none",  # Hide legend for individual plots
        axis.title = element_text(size = rel(FONT_MULTIPLIER)),
        axis.text = element_text(size = rel(FONT_MULTIPLIER)),
        plot.title = element_text(size = rel(1.1 * FONT_MULTIPLIER)),
        axis.text.x = element_blank()  # Hide x-axis text
      )

    # Add y-axis limits for consistent axis across all plots
    violin_plots[[label]] <- violin_plot_obj_for_combined + ylim(range(all_values, na.rm = TRUE))
  }

  # Create a separate legend plot by using a dummy bar plot to generate the legend
  legend_plot <- ggplot(combined_data, aes(x = LLM, fill = LLM)) +
    geom_bar(stat = "count", width = 0, alpha = 0.5) +
    scale_fill_manual(values = LLM_COLOR_MAP) +
    theme_void() +
    theme(
      legend.position = "bottom",  # Place the legend at the bottom
      legend.box = "horizontal",
      legend.title = element_blank(),
      legend.text = element_text(size = rel(FONT_MULTIPLIER)),
    ) +
    scale_fill_manual(
      values = LLM_COLOR_MAP,
      labels = function(x) paste0(x, "   ")  # Add spaces after each legend label
    ) +
    guides(fill = guide_legend(ncol = 7))  # Ensure the legend wraps to a new row



  # Arrange all violin plots in a 1-row grid with consistent y-axis limits
  combined_violin_plot <- plot_grid(
    violin_plots$short, violin_plots$medium, violin_plots$long,
    nrow = 1,
    labels = NULL,  # No labels for the plots
    rel_widths = c(1, 1, 1)  # Equal width for each plot
  )

  # Add the legend plot below the combined violin plot
  combined_violin_plot_with_legend <- plot_grid(
    combined_violin_plot, legend_plot,
    nrow = 2,
    rel_heights = c(9, 1)  # Adjust the height to make room for the legend at the bottom
  )

  # Save the combined violin plot with legend at the bottom
  ggsave(paste0(violin_dir, "/combined_violin_plots_llms_", metric_name, ".pdf"),
         combined_violin_plot_with_legend,
         width = WIDTH * length(LENGTH_MAP), height = HEIGHT)
}

# Call the function to generate violin plots for energy usage
generate_violin_plots_each_llm(filtered_llm_data, ENERGY)


Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Warning message:
“Removed 192 rows containing missing values or values outside the scale range
(`geom_violin()`).”
Warning message:
“Removed 145 rows containing missing values or values outside the scale range
(`geom_violin()`).”


### QQ Plots

In [98]:
generate_qq_plots_for_metric <- function(metric_name) {
  # Function to create QQ plot
  create_qq_plot <- function(data, title) {
    qqPlot(data, main = title, ylab = AXIS_LABELS[[metric_name]],
           col = "blue", col.lines = "red", pch = 16, cex = 0.7)
  }

  # Loop over methods (on-device and remote)
  for (method in c(ON_DEVICE, REMOTE)) {
    # Set up directory for the specific metric and method
    qq_dir <- file.path(QQ_FOLDER, tolower(method), metric_name)
    dir.create(qq_dir, recursive = TRUE, showWarnings = FALSE)

    # Loop over content lengths
    for (label in names(LENGTH_MAP)) {
      length_value <- LENGTH_MAP[[label]]
      subset_name <- paste(method, label, sep = "_")

      # Get the data for this subset
      data <- subset_data[[subset_name]][[metric_name]]

      # Create and save the QQ plot
      plot_title <- paste(tools::toTitleCase(method), " - ", tools::toTitleCase(label),
                    " (", length_value, ")", sep = "")
      pdf(file.path(qq_dir, paste0("qq_plot_", label, ".pdf")),
          width = WIDTH, height = HEIGHT)
      create_qq_plot(data, plot_title)
      dev.off()
    }
  }
}

In [99]:
# Loop through each metric and generate qq plots
for (metric_name in METRICS) {
  generate_qq_plots_for_metric(metric_name)
}

## Normality Testing Using Shapiro-Wilks

### Shapiro-Wilks Test

* A low p-value (typically less than 0.05) indicates that the data significantly deviates from a normal distribution, providing evidence to reject the null hypothesis. In other words, it suggests non-normality.
* It is calculated during the Shapiro-Wilk test and is used to assess the degree of normality in the data. The value of W ranges between 0 and 1, where 1 indicates perfect normality.
* The smaller the W statistic, the stronger the evidence against the null hypothesis (the assumption that the data follows a normal distribution). Smaller W values suggest departures from normality.



In [19]:
# Initialize an empty data frame to store results
shapiro_results_df <- data.frame(
  subset_name = character(),
  W = numeric(),
  p_value = numeric(),
  stringsAsFactors = FALSE
)

# Loop through each subset and perform the Shapiro-Wilk test
for (subset_name in names(subset_data)) {
  data_subset <- subset_data[[subset_name]]

  # Perform the Shapiro-Wilk test
  test_result <- shapiro.test(data_subset[[ENERGY]])

  # Append results to the data frame
  shapiro_results_df <- rbind(shapiro_results_df, data.frame(
    subset_name = subset_name,
    W = test_result$statistic,
    p_value = test_result$p.value,
    stringsAsFactors = FALSE
  ))
}

# Print the results as a table
print(shapiro_results_df)


        subset_name         W      p_value
W   on_device_short 0.9547488 3.202963e-05
W1 on_device_medium 0.8745747 3.521535e-11
W2   on_device_long 0.9290627 5.084372e-08
W3     remote_short 0.8594111 1.116692e-11
W4    remote_medium 0.9094642 2.101168e-08
W5      remote_long 0.9322779 6.199450e-07


## Find skewed data, apply transformation (if applicable), and check if normality is enhanced

In [20]:
# Function to check skewness
check_skew <- function(skew) {
  if (skew > 0) {
    return("Positively Skewed")
  } else if (skew < 0) {
    return("Negatively Skewed")
  } else {
    return("Symmetric")
  }
}

# Function to transform data towards normality and check normality
transform_pairs_towards_normality <- function(on_device, remote, name) {
  # Calculate skewness of datasets
  on_device_skew <- skewness(on_device)
  remote_skew <- skewness(remote)

  results_skew_on_device <- check_skew(on_device_skew)
  results_skew_remote <- check_skew(remote_skew)

  # Print skewness results
  cat(name, " on_device: ", results_skew_on_device, "\n")
  cat(name, " remote: ", results_skew_remote, "\n")

  # Prepare a data frame to store results
  results_df <- data.frame(
    Transformation = character(),
    Shapiro_on_device_p_value = numeric(),
    Shapiro_Remote_p_value = numeric(),
    stringsAsFactors = FALSE
  )

  # Check skewness and apply transformations
  if (results_skew_on_device == results_skew_remote) {
    if (results_skew_on_device == "Positively Skewed") {
      # Transformations for positive skew
      on_device_sqrt <- sqrt(on_device)
      remote_sqrt <- sqrt(remote)
      on_device_log <- log(on_device)
      remote_log <- log(remote)

      # Test normality differences
      shapiro_on_device <- shapiro.test(on_device)
      shapiro_remote <- shapiro.test(remote)
      shapiro_on_device_sqrt <- shapiro.test(on_device_sqrt)
      shapiro_remote_sqrt <- shapiro.test(remote_sqrt)
      shapiro_on_device_log <- shapiro.test(on_device_log)
      shapiro_remote_log <- shapiro.test(remote_log)

      # Store results in data frame
      results_df <- rbind(results_df, data.frame(
        Transformation = "Original",
        Shapiro_on_device_p_value = shapiro_on_device$p.value,
        Shapiro_Remote_p_value = shapiro_remote$p.value
      ))
      results_df <- rbind(results_df, data.frame(
        Transformation = "Square Root",
        Shapiro_on_device_p_value = shapiro_on_device_sqrt$p.value,
        Shapiro_Remote_p_value = shapiro_remote_sqrt$p.value
      ))
      results_df <- rbind(results_df, data.frame(
        Transformation = "Logarithm",
        Shapiro_on_device_p_value = shapiro_on_device_log$p.value,
        Shapiro_Remote_p_value = shapiro_remote_log$p.value
      ))

    } else {
      # Transformations for negative skew
      on_device_power2 <- on_device^2
      remote_power2 <- remote^2
      on_device_power3 <- on_device^3
      remote_power3 <- remote^3

      # Test normality differences
      shapiro_on_device <- shapiro.test(on_device)
      shapiro_remote <- shapiro.test(remote)
      shapiro_on_device_power2 <- shapiro.test(on_device_power2)
      shapiro_remote_power2 <- shapiro.test(remote_power2)
      shapiro_on_device_power3 <- shapiro.test(on_device_power3)
      shapiro_remote_power3 <- shapiro.test(remote_power3)

      # Store results in data frame
      results_df <- rbind(results_df, data.frame(
        Transformation = "Original",
        Shapiro_on_device_p_value = shapiro_on_device$p.value,
        Shapiro_Remote_p_value = shapiro_remote$p.value
      ))
      results_df <- rbind(results_df, data.frame(
        Transformation = "Power 2",
        Shapiro_on_device_p_value = shapiro_on_device_power2$p.value,
        Shapiro_Remote_p_value = shapiro_remote_power2$p.value
      ))
      results_df <- rbind(results_df, data.frame(
        Transformation = "Power 3",
        Shapiro_on_device_p_value = shapiro_on_device_power3$p.value,
        Shapiro_Remote_p_value = shapiro_remote_power3$p.value
      ))
    }
  } else {
    print("on_device and remote datasets are not skewed similarly, so no transformation is applied.")
  }

  return(results_df)
}

# Collect results for each dataset
final_results <- data.frame()

for (length_label in names(LENGTH_MAP)) {  # Iterate over length labels
  for (method in c(ON_DEVICE, REMOTE)) {
    subset_name <- paste(method, length_label, sep = "_")  # Construct subset name
    if (subset_name %in% names(subset_data)) {
      data_subset <- subset_data[[subset_name]]
      if (method == ON_DEVICE) {
        results <- transform_pairs_towards_normality(data_subset[[ENERGY]],
                                                      subset_data[[paste(REMOTE, length_label, sep = "_")]][[ENERGY]],
                                                      length_label)
      }
      final_results <- rbind(final_results, results)
    }
  }
}

# Print the final results table
print(final_results)

short  on_device:  Negatively Skewed 
short  remote:  Positively Skewed 
[1] "on_device and remote datasets are not skewed similarly, so no transformation is applied."
medium  on_device:  Negatively Skewed 
medium  remote:  Negatively Skewed 
long  on_device:  Negatively Skewed 
long  remote:  Negatively Skewed 
   Transformation Shapiro_on_device_p_value Shapiro_Remote_p_value
1        Original              3.521535e-11           2.101168e-08
2         Power 2              7.749464e-10           4.230615e-09
3         Power 3              1.663134e-15           4.418860e-14
4        Original              3.521535e-11           2.101168e-08
5         Power 2              7.749464e-10           4.230615e-09
6         Power 3              1.663134e-15           4.418860e-14
7        Original              5.084372e-08           6.199450e-07
8         Power 2              3.038347e-09           8.610354e-08
9         Power 3              1.149680e-14           9.411754e-11
10       Origina

## H1. Hypothesis Testing and Effect Size Estimation

In [21]:
# Initialize a data frame to store results
results <- data.frame(
  Content_Length = character(),
  W_Value = numeric(),
  P_Value = numeric(),
  Cliffs_Delta = numeric(),
  Lower_CI = numeric(),
  Upper_CI = numeric(),
  Delta = character(),
  stringsAsFactors = FALSE
)

# Loop over content lengths and calculate the statistics using subset_data
for (label in names(LENGTH_MAP)) {
  length_value <- LENGTH_MAP[[label]]

  # Get filtered data from subset_data
  on_device_data <- subset_data[[paste(ON_DEVICE, label, sep = "_")]][[ENERGY]]
  remote_data <- subset_data[[paste(REMOTE, label, sep = "_")]][[ENERGY]]

  # Perform the Wilcoxon test
  wilcoxon_result <- wilcox.test(on_device_data, remote_data, alternative = "two.sided")

  # Perform the Cliff's delta test
  cliff_result <- cliff.delta(on_device_data, remote_data, return.ci = TRUE)

  # Append results to the data frame
  results <- results %>% add_row(
    Content_Length = sprintf("%s (%d words)", tools::toTitleCase(label), length_value),
    W_Value = wilcoxon_result$statistic,
    P_Value = wilcoxon_result$p.value,
    Cliffs_Delta = cliff_result$estimate,
    Lower_CI = cliff_result$conf.int[1],
    Upper_CI = cliff_result$conf.int[2],
    Delta = case_when(
      abs(cliff_result$estimate) < 0.147 ~ "Negligible",
      abs(cliff_result$estimate) < 0.33 ~ "Small",
      abs(cliff_result$estimate) < 0.474 ~ "Medium",
      TRUE ~ "Large"
    )
  )
}

# Prepare LaTeX table
cat("\\begin{table}[H]\n")
cat("  \\centering\n")
cat("  \\caption{Results for Hypothesis 1 - Energy Usage Difference Between On-device and Remote LLMs Content Fetching}\n")
cat("  \\resizebox{\\columnwidth}{!}{%\n")
cat("    \\begin{tabular}{lcccccc}\n")
cat("      \\hline\n")
cat("      \\textbf{Content Length} & \\textbf{W-Value} & \\textbf{P-Value} & \\textbf{Cliff’s Delta} & \\textbf{Lower CI} & \\textbf{Upper CI} & \\textbf{Delta} \\\\\n")
cat("      \\hline\n")

# Loop through results to create table rows
for (i in 1:nrow(results)) {
  cat(sprintf("      \\textbf{%s} & %.0f & %s & %.3f & %.3f & %.3f & %s \\\\\n",
              results$Content_Length[i],
              ifelse(is.na(results$W_Value[i]), NA, results$W_Value[i]),
              ifelse(is.na(results$P_Value[i]), "NA", ifelse(results$P_Value[i] < 0.001, "< 2.2e-16", sprintf("%.1e", results$P_Value[i]))),
              ifelse(is.na(results$Cliffs_Delta[i]), NA, results$Cliffs_Delta[i]),
              ifelse(is.na(results$Lower_CI[i]), NA, results$Lower_CI[i]),
              ifelse(is.na(results$Upper_CI[i]), NA, results$Upper_CI[i]),
              ifelse(is.na(results$Delta[i]), NA, results$Delta[i]))
  )
  cat("      \\hline\n")
}

cat("    \\end{tabular}%
}\n")
cat("  \\label{table:h1}\n")
cat("\\end{table}\n")

\begin{table}[H]
  \centering
  \caption{Results for Hypothesis 1 - Energy Usage Difference Between On-device and Remote LLMs Content Fetching}
  \resizebox{\columnwidth}{!}{%
    \begin{tabular}{lcccccc}
      \hline
      \textbf{Content Length} & \textbf{W-Value} & \textbf{P-Value} & \textbf{Cliff’s Delta} & \textbf{Lower CI} & \textbf{Upper CI} & \textbf{Delta} \\
      \hline
      \textbf{Short (100 words)} & 28370 & < 2.2e-16 & 0.941 & 0.905 & 0.964 & Large \\
      \hline
      \textbf{Medium (500 words)} & 28486 & < 2.2e-16 & 0.956 & 0.920 & 0.976 & Large \\
      \hline
      \textbf{Long (1000 words)} & 29587 & < 2.2e-16 & 0.912 & 0.868 & 0.942 & Large \\
      \hline
    \end{tabular}%
}
  \label{table:h1}
\end{table}


# H2. Scatter Plots

In [22]:
make_scatter_plot_with_correlation <- function(dataset, x_var, y_var, title, x_title, y_title, color) {
  ggplot(dataset, aes(x = !!sym(x_var), y = !!sym(y_var))) +
    geom_point(color = "black", size=0.5) +  # Set points color to black
  geom_smooth(method = "lm", se = FALSE, color = color) +  # Add correlation line with increased thickness
    theme_minimal() +
    theme(
      plot.title = element_text(size = 4 * FONT_MULTIPLIER),
      axis.title.x = element_text(size = 4 * FONT_MULTIPLIER),
      axis.title.y = element_text(size = 4 * FONT_MULTIPLIER),
      axis.text.x = element_text(size = 4 * FONT_MULTIPLIER),
      axis.text.y = element_text(size = 4 * FONT_MULTIPLIER)
    ) +
    labs(
      title = title,
      x = x_title,
      y = y_title
    )
}

In [152]:
# Loop through all metrics to create combined plots
for (metric in METRICS[-1]) {  # Exclude ENERGY from METRICS
  # List to store plots for each length and method
  plot_list <- list()

  # Create a directory for the scatter plots if it does not exist
  scatter_dir <- SCATTER_FOLDER
  if (!dir.exists(scatter_dir)) {
    dir.create(scatter_dir, recursive = TRUE)
  }

  # Iterate over each length label and method
  for (method in c(ON_DEVICE, REMOTE)) {
    for (label in names(LENGTH_MAP)) {
      length_value <- LENGTH_MAP[[label]]
      subset_name <- paste(method, label, sep = "_")  # Construct subset name

      if (subset_name %in% names(subset_data)) {
        data_subset <- subset_data[[subset_name]]

        # Create scatter plot
        plot_title <- paste(tools::toTitleCase(method), " - ", tools::toTitleCase(label), " (", length_value, ")", sep = "")
        x_title <- AXIS_LABELS[[ENERGY]]
        y_title <- AXIS_LABELS[[metric]]
        color <- COLOR_MAP[[method]]  # Set color based on method

        scatter_plot <- make_scatter_plot_with_correlation(
          data_subset, ENERGY, metric, plot_title, ifelse(method == REMOTE, x_title, ""),
          ifelse(label == "short", y_title, ""), color
        ) # only include the x_title and y_title for left-most and bottom subplot

        plot_list[[paste(method, label, sep = "_")]] <- scatter_plot  # Store plot in list
      }
    }
  }

  # Combine plots into a grid for both on-device and remote
  combined_scatter_plot <- plot_grid(
    plot_list[[paste(ON_DEVICE, "short", sep = "_")]],
    plot_list[[paste(ON_DEVICE, "medium", sep = "_")]],
    plot_list[[paste(ON_DEVICE, "long", sep = "_")]],
    plot_list[[paste(REMOTE, "short", sep = "_")]],
    plot_list[[paste(REMOTE, "medium", sep = "_")]],
    plot_list[[paste(REMOTE, "long", sep = "_")]],
    nrow = 2,
    labels = c("A", "B", "C", "D", "E", "F"),
    align = 'hv'
  )

  # Save the combined plot to a PDF file
  ggsave(filename = paste0(scatter_dir, "/", metric, "_vs_", ENERGY, ".pdf"),
         plot = combined_scatter_plot, width = WIDTH, height = WIDTH * 2 / 3)
}


`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


# H2. Spearman

In [24]:
# Function to calculate and format Spearman correlations as LaTeX code
calculate_spearman_latex <- function(subset_data) {

  # Initialize an empty string to store LaTeX table rows
  latex_rows <- ""

  # Loop over methods (ON_DEVICE and REMOTE)
  for (method in c(ON_DEVICE, REMOTE)) {
    method_rows <- ""

    # Loop over content length categories (short, medium, long)
    for (label in names(LENGTH_MAP)) {
      subset_name <- paste(method, label, sep = "_")

      if (subset_name %in% names(subset_data)) {
        data_subset <- subset_data[[subset_name]]

        # Start row with content length label
        row_label <- paste0("& \\textbf{", tools::toTitleCase(label), " (", LENGTH_MAP[[label]], ")} & ")

        # Container for correlation results
        metric_results <- c()

        # Iterate through each metric to correlate with energy usage
        for (metric in c(TIME, CPU, GPU, MEMORY)) {
          # Calculate Spearman correlation and p-value
          spearman_test <- cor.test(data_subset[[ENERGY]], data_subset[[metric]], method = "spearman")
          spearman_coef <- spearman_test$estimate
          p_value <- spearman_test$p.value

          # Format significance stars based on p-value
          significance <- ifelse(p_value < 0.001, "***",
                                 ifelse(p_value < 0.01, "**",
                                        ifelse(p_value < 0.05, "*", "")))

          # Format coefficient and p-value
          formatted_coef <- formatC(spearman_coef, digits = 3, format = "f")
          formatted_p_value <- formatC(p_value, digits = 3, format = "f")

          # Append formatted result for the metric
          formatted_p_value <- if (p_value < 0.001) {
            "<0.001"
          } else {
            formatC(p_value, digits = 3, format = "f")
          }

          # Append formatted result for the metric
          formatted_result <- if (p_value < 0.05) {
            paste0("\\textbf{", formatted_coef, " (", formatted_p_value, significance, ")}")
          } else {
            paste0(formatted_coef, " (", formatted_p_value, significance, ")")
          }

          metric_results <- c(metric_results, formatted_result)


        }

        # Concatenate row with all metrics
        latex_row <- paste(row_label, paste(metric_results, collapse = " & "), "\\\\ \\cline{2-6}\n")
        method_rows <- paste(method_rows, latex_row, sep = "")
      }
    }

    # Add multirow for method (ON_DEVICE or REMOTE)
    multirow_start <- paste0("\\multirow{3}{*}{\\textbf{", tools::toTitleCase(gsub("_", "-", method)), "}} ")
    method_rows <- paste(multirow_start, method_rows)
    latex_rows <- paste(latex_rows, method_rows, "\\hline\n", sep = "")
  }

  # LaTeX table header and footer
  latex_table <- paste0("\\begin{table*}[htbp]\n",
                        "    \\centering\n",
                        "    \\caption{Spearman Rank Correlation for Hypothesis 2: Energy Usage vs. Performance Metrics. \n",
                        "        Significance levels:\\\\\n",
                        "        * < 0.05, ** < 0.01, *** < 0.001 (\\textbf{bold} for significant p-values).\n",
                        "    }\n",
                        "    \\scalebox{0.8}{\n",
                        "    \\begin{tabular}{|l|l|c|c|c|c|}\n",
                        "        \\hline\n",
                        "        \\multirow{1}{*}{\\textbf{Treatment}} & \\multirow{1}{*}{\\textbf{Content length}} & \\multicolumn{1}{c|}{\\textbf{Energy usage vs Execution time}} & \\multicolumn{1}{c|}{\\textbf{Energy usage vs CPU usage}} & \\multicolumn{1}{c|}{\\textbf{Energy usage vs GPU usage}} & \\multicolumn{1}{c|}{\\textbf{Energy usage vs Memory usage}} \\\\\n",
                        "        \\hline\n",
                        latex_rows,
                        "    \\end{tabular}\n",
                        "    }\n",
                        "    \\label{tab:spearman}\n",
                        "\\end{table*}")

  # Print or return the final LaTeX code
  cat(latex_table)
}

# Call the function to print the LaTeX table
calculate_spearman_latex(subset_data)


Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data_subset[[ENERGY]], data_subset[[metric]], :
“Cannot compute exact p-value with ties”


\begin{table*}[htbp]
    \centering
    \caption{Spearman Rank Correlation for Hypothesis 2: Energy Usage vs. Performance Metrics. 
        Significance levels:\\
        * < 0.05, ** < 0.01, *** < 0.001 (\textbf{bold} for significant p-values).
    }
    \scalebox{0.8}{
    \begin{tabular}{|l|l|c|c|c|c|}
        \hline
        \multirow{1}{*}{\textbf{Treatment}} & \multirow{1}{*}{\textbf{Content length}} & \multicolumn{1}{c|}{\textbf{Energy usage vs Execution time}} & \multicolumn{1}{c|}{\textbf{Energy usage vs CPU usage}} & \multicolumn{1}{c|}{\textbf{Energy usage vs GPU usage}} & \multicolumn{1}{c|}{\textbf{Energy usage vs Memory usage}} \\
        \hline
\multirow{3}{*}{\textbf{On-Device}}  & \textbf{Short (100)} &  \textbf{0.991 (<0.001***)} & \textbf{-0.708 (<0.001***)} & \textbf{-0.176 (0.023*)} & \textbf{0.541 (<0.001***)} \\ \cline{2-6}
& \textbf{Medium (500)} &  \textbf{0.928 (<0.001***)} & \textbf{-0.284 (<0.001***)} & \textbf{0.153 (0.039*)} & \textbf{0.520 (<0.001***)} \\ 

# [Only on Google Colab] Download the Visualization Folders

In [153]:
zip("all_folders.zip", files = unlist(lapply(PLOT_FOLDERS, function(folder) {
  list.files(folder, full.names = TRUE, recursive = TRUE)
})))



In [ ]:
# Then, manually download the exported .zip